# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import utils
import pathing

import osmnx as ox
import networkx
import random
import time
from tqdm import tqdm


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Methods

In [5]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


In [6]:
def gen_linkers(road_type_factors, link_factor):
    linkables = [
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
    ]

    for linkable in linkables:
        if linkable in road_type_factors:
            road_type_factors[linkable + "_link"] = road_type_factors[linkable] * link_factor
    
    return road_type_factors


In [14]:
def evaluate_travel_time_accuracy(benchmark_times, calculated_times_with_acute_speeds, calculated_times_without_acute_speeds):
    total_diff = 0
    total_diff_percentage = 0

    for path, acceptable_range in benchmark_times.items():
        diff = abs(calculated_times_without_acute_speeds[path] - calculated_times_with_acute_speeds[path])
        diff_percentage = (diff / calculated_times_with_acute_speeds[path]) * 100

        total_diff += diff
        total_diff_percentage += diff_percentage

        print(f"Path {path}:")
        print(f"  Benchmark Time: {acceptable_range} minutes")
        print(f"  Calculated Time With Acute Speeds: {calculated_times_with_acute_speeds[path]} minutes")
        print(f"  Calculated Time Without Acute Speeds: {calculated_times_without_acute_speeds[path]} minutes")
        print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
        print()

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    print(f"Total Discrepancy: {total_diff} minutes across all paths.")
    print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")


In [30]:
def evaluate_travel_time_accuracy_2(benchmark_times, calculated_times, acute_factor):
    total_diff = 0
    total_diff_percentage = 0

    for path, acceptable_range in benchmark_times.items():
        diff = abs(calculated_times[path] - calculated_times[path] * acute_factor)
        diff_percentage = (diff / calculated_times[path] * acute_factor) * 100

        total_diff += diff
        total_diff_percentage += diff_percentage

        print(f"Path {path}:")
        print(f"  Benchmark Time: {acceptable_range} minutes")
        print(f"  Calculated Time With Acute Speeds: {calculated_times[path] * acute_factor} minutes")
        print(f"  Calculated Time Without Acute Speeds: {calculated_times[path]} minutes")
        print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
        print()

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    print(f"Total Discrepancy: {total_diff} minutes across all paths.")
    print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")


In [31]:
def calculate_error_old(od: pathing.OriginDestination, benchmark_times):
    calculated_times = {}
    
    # Assuming utils and get_node functions are defined and work as intended
    for ((start_lon, start_lat), (end_lon, end_lat)), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat)

        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time

    # Calculate MSE
    errors = []

    for path, acceptable_range in benchmark_times.items():
        calculated_time = calculated_times[path]
        lower_bound, upper_bound = acceptable_range

        # Check if the calculated time falls within the acceptable range
        if lower_bound <= calculated_time <= upper_bound:
            error = 0
        else:
            # If outside the range, find the smallest absolute difference to the bounds
            error = min(abs(calculated_time - lower_bound), abs(calculated_time - upper_bound))
        
        errors.append(error)

    mse = sum(error ** 2 for error in errors) / len(errors)
    
    return mse, calculated_times


In [15]:
def calculate_error(od: pathing.OriginDestination, benchmark_times, current_solution, acute_factor):
    calculated_times_with_acute_speeds = {}
    calculated_times_without_acute_speeds = {}
    
    # Assuming utils and get_node functions are defined and work as intended
    for ((start_lon, start_lat), (end_lon, end_lat)), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat)

        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times_with_acute_speeds[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time
    
    
    od.set_graph_weights_v2(**current_solution, use_ambulance_speeds=False)

    # Assuming utils and get_node functions are defined and work as intended
    for ((start_lon, start_lat), (end_lon, end_lat)), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat)

        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times_without_acute_speeds[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time * acute_factor

    # Calculate MSE
    errors = []

    for path, _ in benchmark_times.items():
        error = abs(calculated_times_without_acute_speeds[path] - calculated_times_with_acute_speeds[path])
        errors.append(error)

    mse = sum(error ** 2 for error in errors) / len(errors)
    
    return mse, calculated_times_with_acute_speeds, calculated_times_without_acute_speeds


In [24]:
def optimize_parameters(od: pathing.OriginDestination, benchmark_times, max_duration_minutes):
    start_time = time.time()
    max_duration_seconds = max_duration_minutes * 60
    
    # Define parameter bounds
    param_bounds = {
        "acute_factor": (0.1, 2.0),
    }
    
    # Initialize the best solution
    best_solution = None
    best_error = float("inf")
    
    # Progress bar
    with tqdm(desc="Optimizing Parameters") as pbar:
        while time.time() - start_time < max_duration_seconds:
            # Generate a random solution within the bounds
            current_solution = {
                "default_intersection_penalty": 4,
                "traffic_signal_penalty": 9,
                "road_type_factors": {
                    "motorway": 0.9004725950459835,
                    "trunk": 0.9602555300158857,
                    "primary": 0.8957176686384549,
                    "secondary": 0.812242943390013,
                    "tertiary": 0.8779790728890131,
                    "unclassified": 0.7187557285185038,
                    "residential": 0.5840639670583821,
                    "living_street": 0.3375975835998158,
                },
            }
            acute_factor = random.uniform(*param_bounds["acute_factor"])

            current_solution["road_type_factors"] = gen_linkers(
                current_solution["road_type_factors"],
                link_factor=0.8521506076850045
            )
            
            # Set graph weights with the current solution
            od.set_graph_weights_v2(**current_solution, use_ambulance_speeds=True)
            
            # Calculate the error for the current solution
            current_error, _, _ = calculate_error(od, benchmark_times, current_solution, acute_factor)
            
            # Update best solution if the current one is better
            if current_error < best_error:
                best_solution = current_solution
                best_acute_factor = acute_factor
                best_error = current_error
                tqdm.write(
                    f"    New best solution:\n"
                    f"        Triage A Factor: {best_acute_factor}\n"
                    f"        Error: {best_error}"
                )
            else:
                if acute_factor > best_acute_factor:
                    param_bounds["acute_factor"] = (param_bounds["acute_factor"][0], acute_factor)
                else:
                    param_bounds["acute_factor"] = (acute_factor, param_bounds["acute_factor"][1])
                print(param_bounds["acute_factor"])
            
            # Update progress bar
            pbar.update()

            # early stop
            if current_error == 0:
                break
    
    return best_solution, best_acute_factor, best_error


In [10]:
def fix_lat_lon(benchmark_times):
    new_benchmark_times = {}

    for ((start_lat, start_lon), (end_lat, end_lon)), benchmark in benchmark_times.items():
        new_benchmark_times[((start_lon, start_lat), (end_lon, end_lat))] = benchmark

    return new_benchmark_times

# Main

In [11]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
od.node_validator = ox.distance.nearest_nodes(od.graph, central_depot_x, central_depot_y)


In [25]:
# times from Google Maps (17/03/2024 02:00)
benchmark_times = {
    ((59.9369806, 10.7343661), (59.6800045, 10.6572104)): (35, 45),
    ((59.93596, 10.73244), (59.92629, 10.77570)): (4, 9),
    ((59.90864, 10.73921), (59.93037, 10.77273)): (9, 16),
    ((59.92727, 10.73174), (59.86305, 10.66617)): (50, 60),
    ((59.82052, 10.47168), (59.95577, 11.04773)): (40, 50),
    ((60.00352, 10.76216), (59.83905, 10.80742)): (28, 28),

    ((60.31262, 11.14674), (60.11964, 11.47577)): (40, 40),
    ((60.11964, 11.47577), (60.13423, 11.166858)): (22, 26),
    ((60.13423, 11.166858), (59.879044, 11.561687)): (50, 50),
    ((59.879044, 11.561687), (59.932076, 10.987775)): (40, 40),
    ((59.932076, 10.987775), (60.042152, 10.880784)): (20, 22),
    ((60.042152, 10.880784), (59.930733, 10.831094)): (20, 20),
    ((59.930733, 10.831094), (59.917, 10.758744)): (9, 14),
    ((59.917, 10.758744), (59.93917, 10.741926)): (7, 14),
    ((59.93917, 10.741926), (59.71566, 10.853331)): (35, 35),
    ((59.71566, 10.853331), (59.659687, 10.725881)): (18, 18),
    ((59.659687, 10.725881), (59.833073, 10.806946)): (22, 22),
    ((59.833073, 10.806946), (59.830055, 10.439887)): (30, 40),
    ((59.830055, 10.439887), (59.89813, 10.509723)): (14, 14),
    ((59.89813, 10.509723), (59.939663, 10.687367)): (16, 20),
    ((59.939663, 10.687367), (59.893173, 10.806364)): (12, 16),
    ((59.893173, 10.806364), (59.960304, 10.884091)): (12, 18),
    ((59.960304, 10.884091), (59.997875, 11.03928)): (20, 20),
    ((59.997875, 11.03928), (59.917873, 10.585751)): (28, 35),
    ((59.917873, 10.585751), (60.31262, 11.14674)): (55, 65),
}

benchmark_times = fix_lat_lon(benchmark_times)


In [28]:
best_solution, best_acute_factor, best_error = optimize_parameters(
    od,
    benchmark_times,
    max_duration_minutes=20
)


Optimizing Parameters: 0it [00:00, ?it/s]

Optimizing Parameters: 1it [00:18, 18.19s/it]

    New best solution:
        Triage A Factor: 1.6022459604649324
        Error: 609.7329069173169


Optimizing Parameters: 2it [00:36, 18.11s/it]

    New best solution:
        Triage A Factor: 0.5558308717442332
        Error: 54.695341600307785


...

(0.7953593884696724, 0.7954134098647235)


Optimizing Parameters: 53it [19:40, 24.08s/it]

(0.7953593884696724, 0.7954081367773088)


Optimizing Parameters: 54it [20:02, 22.26s/it]

(0.7953593884696724, 0.7953829926922608)


In [29]:
od.set_graph_weights_v2(**best_solution, use_ambulance_speeds=True)

_, calculated_times_with_acute_speeds, calculated_times_without_acute_speeds = calculate_error(od, benchmark_times, best_solution, best_acute_factor)

evaluate_travel_time_accuracy(benchmark_times, calculated_times_with_acute_speeds, calculated_times_without_acute_speeds)


Path ((10.7343661, 59.9369806), (10.6572104, 59.6800045)):
  Benchmark Time: (35, 45) minutes
  Calculated Time With Acute Speeds: 32.76485672202334 minutes
  Calculated Time Without Acute Speeds: 31.461585075283296 minutes
  Difference: 1.3032716467400434 minutes (3.98%)

Path ((10.73244, 59.93596), (10.7757, 59.92629)):
  Benchmark Time: (4, 9) minutes
  Calculated Time With Acute Speeds: 5.914945242980406 minutes
  Calculated Time Without Acute Speeds: 5.455792367342134 minutes
  Difference: 0.4591528756382717 minutes (7.76%)

Path ((10.73921, 59.90864), (10.77273, 59.93037)):
  Benchmark Time: (9, 16) minutes
  Calculated Time With Acute Speeds: 10.43831711996 minutes
  Calculated Time Without Acute Speeds: 8.665639755664445 minutes
  Difference: 1.7726773642955553 minutes (16.98%)

Path ((10.73174, 59.92727), (10.66617, 59.86305)):
  Benchmark Time: (50, 60) minutes
  Calculated Time With Acute Speeds: 40.76299772220561 minutes
  Calculated Time Without Acute Speeds: 42.3905604526

In [32]:
od.set_graph_weights_v2(**best_solution)

_, calculated_times = calculate_error_old(od, benchmark_times)

evaluate_travel_time_accuracy_2(benchmark_times, calculated_times, best_acute_factor)


Path ((10.7343661, 59.9369806), (10.6572104, 59.6800045)):
  Benchmark Time: (35, 45) minutes
  Calculated Time With Acute Speeds: 31.461585075283296 minutes
  Calculated Time Without Acute Speeds: 39.55585198503308 minutes
  Difference: 8.094266909749784 minutes (16.28%)

Path ((10.73244, 59.93596), (10.7757, 59.92629)):
  Benchmark Time: (4, 9) minutes
  Calculated Time With Acute Speeds: 5.455792367342134 minutes
  Calculated Time Without Acute Speeds: 6.859429199999245 minutes
  Difference: 1.403636832657111 minutes (16.28%)

Path ((10.73921, 59.90864), (10.77273, 59.93037)):
  Benchmark Time: (9, 16) minutes
  Calculated Time With Acute Speeds: 8.665639755664445 minutes
  Calculated Time Without Acute Speeds: 10.89508881102026 minutes
  Difference: 2.229449055355815 minutes (16.28%)

Path ((10.73174, 59.92727), (10.66617, 59.86305)):
  Benchmark Time: (50, 60) minutes
  Calculated Time With Acute Speeds: 42.39056045268041 minutes
  Calculated Time Without Acute Speeds: 53.29657519